本文学习于论文《Hierarchical Attention Networks for Document Classification》,作者:Zichao Yang.

# 论文解读

## 摘要: Abstract

我们提出了一个文档分类的**分层注意力网络**，它有**两个鲜明的特点**:(1)它有一个映射**文档层次结构**的**分层结构**;(2)它有**两个层次的注意力机制**，分别作用于**单词层次**和**句子层次**，使其能够在构建文档表示时区别对待**更重要的内容**和**不太重要的内容**。在六个大规模文本分类任务上的实验表明，我们提出的模型优于已知的技术且提升幅度很大。**注意力层的可视化**表明，模型选择了**有定性信息的单词和句子**。

## 引言: Introduction

文本分类是自然语言处理中的基础任务之一，其目标是将标签分派给文本。它有广泛的应用包括主题标签，情感分类和垃圾邮件检测。**传统的文本分类方法用稀疏词汇特征表示文档**，例如　n-grams，然后在此表示上用一个线性模型或核方法。最近的方法**使用深度学习**，例如卷积神经网络和基于ＬＳＴＭ的循环神经网络，**来学习文档表示**。

尽管基于神经网络的文本分类方法非常有效，在这篇论文中，我们检验了**将文档结构知识(knowledge of document structure)融入模型结构能获得更好的文档表示**这个假设。我们模型的直观认识是，并非文档的所有部分都与回答一个查询**同等相关**，确定相关部分涉及对单词相互作用的建模而不仅仅是每个单词的孤立存在。

我们的主要贡献是提出了一个新的神经结构,即**分层注意力网络(ＨＡＮ)**,旨在捕获文档结构的两个基本认知。第一，**由于文档具有层次结构(单词构成句子;句子形成文档),我们同样通过首先构建句子表示，然后将它们聚合成文档表示，来构造一个文档表示**。第二，**观察到文档中的不同单词和句子具有差异性信息**。此外，**单词和句子的重要性高度依赖于语境，即同一个单词或句子在不同的语境中可能具有不同的重要性**。为了包含对这一事实的敏感性，我们的模型包括了**两个层次的注意力机制**:一个是**单词水平的**，一个是**句子水平的**。这样在构建文档表示时，会使模型将更多或者更少的注意力放置于每个单词和每个句子上。

为了说明，考虑下图的例子
![HAN_Example](./image/HAN_Example.png)

这是一个简短的Yelp评论，任务是按照１－５的等级预测评级。直观地说，第一句和第三句在预测评级方面有更强的信息。在这些句子中，单词**delicious**,**amazing**更多地暗示了本评论中所包含的积极态度。

注意力有两个好处:它不仅会带来更好的性能，而且提供了哪些单词和句子对分类决策有贡献的洞察力，这在应用和分析中具有重要价值。

与先前工作的主要区别是，我们的模型**使用上下文来发现何时一个标记序列相关**，而不是**不考虑上下文地简单过滤标记序列**。为了评估我们模型的表现与其他六个分类模型，我们使用了６个数据集。我们的模型比其它模型有很大的提高。

## 分层注意力网络: Hierarchical Attention Networks

分层注意力网络ＨＡＮ的结构如下所示:
![HAN](./image/HAN.png)

它包含了几部分:**一个单词序列的编码器**，**一个单词层次的注意力层**，**一个句子编码器**，**一个句子层次的注意力层**。下面，我们会详细描述这几部分。

### 基于ＧＲＵ的序列编码器: GRU-based sequence encoder

GRU使用**门机制来记录序列的状态**，而不是使用单独的记忆细胞。有两种类型的门:**重置门$r_{t}$**和**更新门$z_{t}$**。它们共同控制信息如何更新到状态。在时刻$t$,ＧＲＵ计算新状态的方式如下:

$$h_{t} = (1 - z_{t})\bigodot h_{t-1} + z_{t}\bigodot \widetilde{h}_{t}, (1) $$

这是前一时刻状态$h_{t-1}$和由新序列信息计算的当前新状态$\widetilde{h}_{t}$的线性插值。**更新门$z_{t}$**决定**保留多少过去的信息，以及增加多少新的信息**。$z_{t}$的更新公式如下:

$$z_{t} = \sigma(W_{z}x_{t} + U_{z}h_{t-1} + b_{z}),(2)$$

$x_{t}$是时刻$t$的序列向量。候选状态$\widetilde{h}_{t}$以类似于传统循环神经网络ＲＮＮ的方式计算:

$$\widetilde{h}_{t} = tanh\left(W_{h}x_{t} + r_{t}\bigodot(U_{h}h_{t-1}) + b_{h} \right),(3)$$

$r_{t}$是重置门，它**控制了过去状态有多少贡献给了候选状态**$\widetilde{h}_{t}$。如果$r_{t}＝０$，那么它会遗忘过去的状态。重置门的计算方式如下:

$$r_{t} = \sigma(W_{r}x_{t} + U_{r}h_{t-1} + b_{r}) ,(4)$$

### 层次注意力: Hierarchical Attention

本论文中，我们关注于文档水平的分类任务。假设文档有$L$个句子$s_{i}$,每个句子包含$T_{i}$个单词。$w_{it},t\in \left[1,T\right]$,表示单词在第$i$个句子中。ＨＡＮ将原始文档映射成一个向量表示，在其上我们构建了一个分类器来分类文档。下面我们将展示**如何使用分层结构在单词向量上逐步构建文档层次的向量**。

#### 单词编码器: Word Encoder

给定单词序列$w_{it},t\in\left[0,T\right]$构成的句子,首先通过一个嵌入矩阵$W_{e}$将单词转为向量，即$x_{ij} = W_{e}w_{ij}$。我们使用一个双向ＧＲＵ，通过从单词序列的两个方向(正反)汇总信息来获取单词的表示(annotation)，因而将上下文信息合并到单词的表示(annotation)中了。双向ＧＲＵ包含了: 正向ＧＲＵ$\overrightarrow{f}$,它从单词$w_{i1}\sim w_{iT}$读取句子$s_{i}$ ; 负向ＧＲＵ$\overleftarrow{f}$,它从单词$w_{iT}\sim w_{i1}$读取句子:

$$ x_{it} = W_{e}w_{it},t\in\left[1,T\right],$$

$$\overrightarrow{h}_{it} = \overrightarrow{GRU}(x_{it}),t\in\left[1,T\right],$$

$$\overleftarrow{h}_{it} = \overleftarrow{GRU}(x_{it}),t\in\left[T,1\right] .$$

我们将正向隐藏状态$\overrightarrow{h}_{it}$和负向隐藏状态$\overleftarrow{h}_{it}$拼接在一起以获得给定单词的表示(annotation),即

$$h_{it} = \left[\overrightarrow{h}_{it},\overleftarrow{h}_{it} \right]$$

它汇聚了以单词$w_{it}$为中心的整个句子的信息。

值得注意的是，我们直接使用了单词的词嵌入。如果想要一个更好的模型，我们可以用ＧＲＵ从特征中直接获取单词向量，类似于Wang Ling他们在论文《Finding function in form: Compositional character models for　open vocabulary word　representation》所做的工作。

#### 词注意力: Word Attention

并非所有单词对**句子含义的表示**都有同等作用。因此，我们引入注意力机制来抽取这样的单词:它们对句子的含义非常重要，然后汇总那些信息词的表示来形成句向量。特别地:

$$u_{it} = tanh\left(W_{w}h_{it} + b_{w}\right),(5)$$

$$\alpha_{it} = \frac{exp\left(u_{it}^{T}u_{w}\right)}{\sum_{t}exp\left(u_{it}^{T}u_{w}\right)},(6)$$

$$s_{i} = \sum_{t}\alpha_{it}h_{it},(7)$$

也就是说，首先，我们将单词的表示$h_{it}$通过一个单层的ＭＬＰ的非线性变换得到$u_{it}$,将其作为$h_{it}$的隐藏表示;然后，我们计算单词作为$u_{it}$与单词级别的上下文向量$u_{w}$的相似度的重要性，然后通过softmax函数获取正则化的重要性权重$\alpha_{it}$。上下文向量$u_{w}$被视为一个固定查询"what is the informative word"在单词上高层次的表示，就像记忆网络中所使用的那样(Sainbayar Sukhbaatar的《End-to-end memory networks》,Ankit Kumar的《Ａsk me anything: Dynamic memory networks for natural language processing》)。单词的上下文向量$u_{w}$是随机初始化的，并且在训练过程中学习。

#### 句子编码器: Sentence Encoder

给定句子向量$s_{i}$,我们可以以类似的方式获得文档向量。我们使用一个双向ＧＲＵ来编码句子:

$$\overrightarrow{h}_{i} = \overrightarrow{GRU}(s_{i}),i\in \left[1,L\right],$$

$$\overleftarrow{h}_{i} = \overleftarrow{GRU}(s_{i}),t\in \left[L,1\right] $$

我们将$\overrightarrow{h}_{i}$和$\overleftarrow{h}_{i}$拼接在一起构成句向量$i$,即

$$h_{i} = \left[\overrightarrow{h}_{i},\overleftarrow{h}_{i}\right]$$

$h_{i}$聚合了句子$i$附近的句子，但仍关注于句子$i$。

#### 句子注意力: Sentence Attention

为了奖励那些正确分类文档的句子，我们再次将注意力机制引入，并且使用一个句子层次的上下文向量$u_{s}$来衡量句子的重要性。计算方式如下:

$$u_{i} = tanh\left(W_{s}h_{i} + b_{s} \right),(8)$$

$$\alpha_{i} = \frac{exp(u_{i}^{T}u_{s})}{\sum_{i}exp(u_{i}^{T}u_{s})},(9)$$

$$v = \sum_{i}\alpha_{i}h_{i},(10)$$

这里$v$是文档向量，而它聚合了文档中所有句子的信息。类似地，句子层次的上下文向量$u_{s}$可以随机初始化，而后在训练过程中与其他参数一块学习。

## 文档分类: Document Classification

文档向量$v$是一个高层次的文档表示，可以被用作文档分类的特征:

$$p = softmax\left(W_{c}v + b_{c} \right),(11)$$

我们使用正确类别的负的对数似然函数作为训练损失:

$$L = -\sum_{d}logp_{dj},(12)$$

这里$j$是文档$d$的标签。

## 模型配置和训练: Model configuration and training

## Context dependent attention weights

## Visualization of attention

## 结论: Conclusion

在本论文中，我们提出了**ＨＡＮ**来分类文档。同时作为模型便利的副产物，我们使用文档高度信息化的成分实现了更好的可视化。我们的**模型通过将重要的单词聚合到句子中，然后将重要的句子向量聚合到文档向量中，来逐步构建文档向量**。实验结果表明，我们的模型比已有模型表现更好。可视化注意力层表明，我们的模型在挑选重要单词和句子时非常有效。

# TensorFlow实现